In [4]:
import torch.nn as nn
from torch_geometric.nn import HeteroConv, Linear

In [ ]:
class HeteroGNN(nn.Module):
    def __init__(self, lv_conv, ll_conv, vv_conv, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = nn.ModuleList()

        self.lv_conv = lv_conv(-1, hidden_channels)
        self.ll_conv = ll_conv(-1, hidden_channels)
        self.vv_conv = vv_conv(-1, hidden_channels)

        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ("local", "to", "virtual"): self.lv_conv,
                ("local", "to", "local"): self.ll_conv,
                ("virtual", "to", "virtual"): self.vv_conv,
            }, aggr="sum")
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        return self.lin(x_dict["local"])